<a href="https://colab.research.google.com/github/RoseSarlake/color-lab/blob/main/homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
def read_envi(fileName):
    """
    Read Specim ENVI data file. Return 3-D spectral data cube and wavelengths.
    
    dataCube, wavelengths = read_envi(fileName)
    
    
    INPUT:      fileName = Name of the ENVI file (without extention)
                           (e.g. 'sample_001')
                         
    OUTPUTS:    dataCube = 3-D data cube containing the spectral bands 
                           along the 3rd axis.
                             
                wavelengths = Wavelengths of the spectral bands.
    """
    
    import numpy as np

    # Read header file: 
    f = open(fileName + '.hdr', 'r')
    header = f.readlines()
    f.close()
    
    for line in header:
        line = line.lower().strip()
        if 'samples' in line:
            samples = int(line.split('samples = ')[1])
        if 'bands' in line and 'default' not in line:
            bands = int(line.split('bands = ')[1])
        if 'lines' in line:
            lines = int(line.split('lines = ')[1])
        if 'interleave' in line:
            interleave = line.split('interleave = ')[1]
    
    wavelengths = ''
    isWavelength = False
    
    for line in header:
        line = line.lower().strip()
        if 'wavelength =' in line:
            isWavelength = True
        if isWavelength:
            wavelengths += line
    
    f1 = wavelengths.find('{')
    f2 = wavelengths.find('}')
    wavelengths = wavelengths[f1+1:f2]
    wavelengths = [float(n) for n in wavelengths.split(',')]
    wavelengths = np.array(wavelengths)
    
    # Read raw data file:
    f = open(fileName + '.raw', 'r')
    raw_data = np.fromfile(f, dtype='uint16')
    f.close()
    
    # Reshape raw data into a 3-D data cube:
    if interleave == 'bil':
        dataCube = raw_data.reshape((lines, bands, samples))
        dataCube = dataCube.swapaxes(1,2)
    elif interleave == 'bip':
        dataCube = raw_data.reshape((lines, samples, bands))
    elif interleave == 'bsq':
        dataCube = raw_data.reshape((bands, lines, samples))
        dataCube = dataCube.swapaxes(0,1)
        dataCube = dataCube.swapaxes(1,2)
    
    dataCube = dataCube.astype('float')
    
    return dataCube, wavelengths


In [15]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
path = "/gdrive/MyDrive/Colab Notebooks/color lab/ColorChecker_Specim_V10E/capture"

Mounted at /content/gdrive


In [16]:
# 0. Import the read function 
import numpy as np
import matplotlib.pyplot as plt

In [17]:
# 1. Read spectral data cubes (sample, white, dark)
sampleCube,wavelengths = read_envi('colorchecker_sample_0007')
white,wavelengths = read_envi('WHITEREF_COLORchecker_sample_0007')
dark,wavelengths = read_envi('DARKREF_COLORchecker_sample_0007')
    #resize the dark cube and white cube 
darkCube = np.zeros((567,1072,135))
for i in range(0,135):
    cur_mean = np.mean(dark[:,:,i])
    darkCube[:,:,i] = cur_mean
    
whiteCube = np.zeros((567,1072,135))
for i in range(0,135):
    cur_mean = np.mean(white[:,:,i])
    whiteCube[:,:,i] = cur_mean

FileNotFoundError: ignored

In [ ]:
# 2. Calculate spectral reflectance image
refCube = np.zeros((567,1072,135))
for x in range(567):
    for y in range(1072):
        for lamda in range(135):
            refCube[x,y,lamda] = (sampleCube[x,y,lamda]-darkCube[x,y,lam

In [ ]:
# 3. Plot some reflectance spectra from Rsample(x,y,λ)
plt.plot(wavelengths,refCube[400,300,:]) # light green
plt.plot(wavelengths,refCube[350,800,:]) # orange
plt.plot(wavelengths,refCube[200,550,:]) # red
plt.plot(wavelengths,refCube[500,300,:]) # light purple
plt.plot(wavelengths,refCube[100,700,:]) # gray


In [ ]:
# 4. Show some spectral band images Rsample(x,y,λ)
plt.gray()
plt.imshow(refCube[:,:,20]) #420.98nm
plt.imshow(refCube[:,:,34]) #489.95nm
plt.imshow(refCube[:,:,58]) # 610.06nm
plt.imshow(refCube[:,:,100]) # 826.01nm

In [ ]:
# 5. Create and show some falsecolor images
    # 630 nm for red, 532 nm for green, and 465 nm for blue
red = refCube[:,:,62] # 630.31nm
green = refCube[:,:,43] #534.71nm
blue = refCube[:,:,29] #465.22nm
rgbImg = np.dstack((red,green,blue))
plt.imshow(rgbImg)
    # pick some random channels
c1 = refCube[:,:,30] #470.16nm
c2 = refCube[:,:,65] #645.55nm
c3 = refCube[:,:,110] #878.5nm
falseColorImg_1 = np.dstack((c1,c2,c3))
plt.imshow(falseColorImg_1)

c4 = refCube[:,:,120] # 931.41nm
c5 = refCube[:,:,20] # 420.98nm
c6 = refCube[:,:,50] #569.76nm
falseColorImg_2 = np.dstack((c4,c5,c6))
plt.imshow(falseColorImg_2)